In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

import numpy as np

temp = []
for img in x_train:
    t = []
    for row in img:
        for i in row:
            t.append(i)
    temp.append(t)
x_train = []
x_train = temp

x_train = np.array(x_train)

x_train = x_train.reshape(60000,784)

model = Sequential()
model.add(Dense(784,activation='relu',input_dim=784))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(784,activation='relu'))

model.compile(loss=keras.losses.mean_squared_error,
             optimizer=keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0),
             metrics = ['accuracy'])

model.fit(x_train,x_train,verbose=1,epochs=20,batch_size=256)

Epoch 1/20
60000/60000 [==============================] - 33s 545us/step - loss: 0.0599 - acc: 0.0104
Epoch 2/20
60000/60000 [==============================] - 30s 497us/step - loss: 0.0384 - acc: 0.0112
Epoch 3/20
60000/60000 [==============================] - 29s 480us/step - loss: 0.0327 - acc: 0.0109
Epoch 4/20
60000/60000 [==============================] - 29s 484us/step - loss: 0.0294 - acc: 0.0110
Epoch 5/20
60000/60000 [==============================] - 29s 484us/step - loss: 0.0273 - acc: 0.0115
Epoch 6/20
60000/60000 [==============================] - 26s 435us/step - loss: 0.0256 - acc: 0.0112
Epoch 7/20
60000/60000 [==============================] - 25s 411us/step - loss: 0.0243 - acc: 0.0112
Epoch 8/20
60000/60000 [==============================] - 30s 498us/step - loss: 0.0234 - acc: 0.0110
Epoch 9/20
60000/60000 [==============================] - 29s 485us/step - loss: 0.0226 - acc: 0.0121
Epoch 10/20
60000/60000 [==============================] - 30s 497us/step - loss: 

In [3]:
weights = []
for layer in model.layers:
    weights.append(layer.get_weights())

In [4]:
model_1 = Sequential()
model_1.add(Dense(784,activation='relu',weights=weights[0],input_dim=784))
model_1.add(Dense(256,activation='relu',weights=weights[1]))
model_1.add(Dense(128,activation='relu',weights=weights[2]))
model_1.add(Dense(64,activation='relu',weights=weights[3]))

print(model_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 784)               615440    
_________________________________________________________________
dense_9 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
Total params: 857,552
Trainable params: 857,552
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
train_features = model_1.predict(x_train)

In [6]:
print(train_features.shape)

(60000, 64)


In [14]:
x_test_new = []
count = 1
for val in x_test:
    temp = []
    for i in val:
        for j in i:
            for k in j:
                temp.append(k)
    count += 1
    x_test_new.append(temp)
x_test_new = np.array(x_test_new)
print(x_test_new.shape)

(10000, 784)


In [15]:
test_features = model_1.predict(x_test_new)

In [24]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC()
clf.fit(train_features,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
pred = clf.predict(test_features)
print(accuracy_score(y_test,pred))

0.9777
